<a href="https://colab.research.google.com/github/meet7515/CE888/blob/main/lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook you'll create your own bootstrap function following the bootstrap algorithm (check the lecture notes!)

In [ ]:
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np



In [ ]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/customers.csv')
data = df.values.T[1]

In [ ]:
# Checking the notes from the lecture, create here your own bootstrap function:
# 1. Sample from the input array x to create an array of samples of shape (n_bootstraps, sample_size)
# Hint: Check the function random.choice() on Numpy
# 2. Calculate and save the mean of the array (this is "data_mean" that is returned by the function)
# 3. Calculate the mean from each bootstrap (i.e., row) and store it.
# (This should be an array of n_bootstraps values)
# 4. Calculate the lower and upper bounds for a 95% CI (hint: check the percentile function on Numpy)
# 5. Return data_mean, and the lower and upper bounds of your interval
def bootstrap_mean(x, sample_size, n_bootstraps):
    sample_array = np.random.choice(x, size=(n_bootstraps, sample_size))
    rows_sum = np.sum(sample_array, axis=1) / sample_size
    data_mean = np.sum(rows_sum) / n_bootstraps
    lower = np.percentile(rows_sum, 2.5)
    upper = np.percentile(rows_sum, 97.5)
    return data_mean, lower, upper


In [ ]:
# Call your bootstrap function and plot the results

boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean(data, data.shape[0], i)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Bootstrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(x=df_boot.columns[0], y=df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 50000)


NameError: ignored


Now, modify the bootstrap function you created above so that you can pass your desired confidence interval as a parameter.



In [ ]:
def bootstrap_mean_ci(sample, sample_size, n_bootstraps, ci):
    sample_array = np.random.choice(sample, size=(n_bootstraps, sample_size))
    rows_sum = np.sum(sample_array, axis=1) / sample_size
    data_mean = np.sum(rows_sum) / n_bootstraps
    lower = np.percentile(rows_sum, (100 - ci) / 2)
    upper = np.percentile(rows_sum, 100 - (100 - ci) / 2)
    return data_mean, lower, upper

In [ ]:
boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean_ci(data, data.shape[0], i, 80)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(x=df_boot.columns[0], y=df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 50000)

#sns_plot.savefig("bootstrap_confidence_80.pdf", bbox_inches='tight')


# Vehicles dataset

Now let's work on a different dataset, which is stored in the vehicles.csv file.


In [ ]:
# Load and visualise the vehicles dataset
# To load the dataset: https://neptune.ai/blog/google-colab-dealing-with-files (check section "Load individual files directly from GitHub")


# Note that the current and new fleets are in different columns and have different lengths, so bear this in mind when you're plotting.
# You can create separate scatterplots for the two fleets, as you would with the histograms, 
# or plot them both in one plot (but not one against the other).
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/vehicles.csv')
curfleet = df['Current fleet'].to_numpy()
newfleet = df['New Fleet'].dropna().to_numpy()

matplotlib.pyplot.scatter([range((len(curfleet)))], curfleet)
matplotlib.pyplot.xlabel('index')
matplotlib.pyplot.ylabel('Current fleet')
# Note: you can add more cells as needed to organise your code and your plots


## Compare the two fleets

The business analysts come up a comparison algorithm that requires the upper and lower bounds for the mean in order to say which fleet is better.
1. Calculate the mean of both samples.
2. Using the bootstrap function that you created:
    - Construct the 95% CI of the mean of the current fleet.
    - Construct the 95% CI of the mean of the new fleet.
    - Are they comparable? (i.e., is one better than the other?) -- you can do this with a permutation test (check the lecture notes!)

In [ ]:
curboots = bootstrap_mean_ci(curfleet, len(curfleet), n_bootstraps=10000, ci=95)
newboots = bootstrap_mean_ci(newfleet, len(newfleet), n_bootstraps=10000, ci=95)
print(curboots)
print(newboots)

(20.147763052208838, 19.365461847389557, 20.967871485943775)
(30.48003670886076, 29.151898734177216, 31.82278481012658)


In [ ]:
from numpy.random import default_rng
# Create your own function for a permutation test here (you will need it for the lab quiz!):
def permut_test(sample1, sample2, n_permutations):
    """
    sample1: 1D array
    sample2: 1D array (note that the size of the two arrays can be different)
    n_permutations: number of permutations to calculate the p-value
    """
    if (len(sample1)==0) or (len(sample2)==0):
      raise ValueError('One of the lists is empty')
    rng = default_rng()
    t_obs = np.mean(sample2) - np.mean(sample1)
    pvalarray = list()
    for _ in range(n_permutations):
        concat = np.concatenate((sample1, sample2))
        perm = rng.permutation(concat)
        t_perm = np.mean(perm[int(len(sample1)):]) - np.mean(perm[:int(len(sample1))])
        if t_perm > t_obs:
            pvalarray.append(1)
    pvalue = len(pvalarray) / n_permutations
    return pvalue